In [11]:
import pandas as pd
import numpy as np
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
from sklearn.metrics import roc_auc_score

# 1. 데이터 로드 및 파생 변수 추가
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

def add_advanced_features(df):
    df['Finishing_Power'] = df['Finishing'] * 0.6 + df['ShotPower'] * 0.4
    df['Offensive_IQ'] = df[['Positioning', 'Vision', 'Reactions']].mean(axis=1)
    df['Physical_Potential'] = df[['Strength', 'Jumping', 'Stamina', 'SprintSpeed']].mean(axis=1)
    df['On_the_Ball'] = df[['Dribbling', 'BallControl', 'Agility', 'Balance']].mean(axis=1)
    df['Clutch_Score'] = df[['Composure', 'Penalties', 'Volleys']].mean(axis=1)
    
    total_skill = df['Finishing_Power'] + df['Offensive_IQ'] + df['On_the_Ball'] + df['Physical_Potential']
    df['Prospect_Index'] = (total_skill * df['Clutch_Score']) / (df['Age'] + 1)
    return df

train_adv = add_advanced_features(train)
test_adv = add_advanced_features(test)

# 2. LightAutoML 설정
# 'binary' 태스크를 정의하고 AUC를 평가지표로 사용합니다.
task = Task('binary', metric='auc')

# 피처 역할 설정 (타겟 컬럼과 드롭할 컬럼 지정)
roles = {
    'target': 'Prospect',
    'drop': ['ID']
}

# 3. 모델 학습 (TabularAutoML)
# timeout은 학습 제한 시간(초)입니다.
automl = TabularAutoML(
    task=task, 
    timeout=300, 
    cpu_limit=4,
    general_params={'use_algos': [['lgb', 'cb']]}, # LightGBM과 CatBoost 위주로 사용
    reader_params={'n_jobs': 4, 'cv': 5, 'random_state': 42}
)

#print("⏳ LightAutoML 학습 및 교차 검증 진행 중...")
oof_pred = automl.fit_predict(train_adv, roles=roles, verbose=1)

# 4. 검증 성능 확인 (OOF 점수)
auc_score = roc_auc_score(train_adv['Prospect'], oof_pred.data[:, 0])
#print(f"\n🔥 LightAutoML 검증 AUC: {auc_score:.4f}")

# 5. 테스트 데이터 예측
test_pred = automl.predict(test_adv)
test_adv['Prospect_Prob'] = test_pred.data[:, 0]

# 6. 공격수 포지션 필터링 및 TOP 10 선정
# 필터링할 공격수 포지션 리스트
fw_positions = ['ST', 'LW', 'RW', 'CF', 'LF', 'RF']

# 필터링 수행
fw_df = test_adv[test_adv['Position'].isin(fw_positions)].copy()

# 데이터가 있는지 확인용 출력
print(f"DEBUG: 필터링된 공격수 수 = {len(fw_df)}명")

if len(fw_df) > 0:
    # 확률 순으로 정렬 후 상위 10명 (데이터가 10명보다 적으면 있는 만큼만 나옴)
    final_list = fw_df.sort_values(by='Prospect_Prob', ascending=False).head(10)
    
    # 7. 결과 출력 및 CSV 저장
    print("\n" + "="*95)
    print("🏆 [LightAutoML 버전] 차세대 공격수 유망주 TOP 10")
    print("-" * 95)
    output_cols = ['ID', 'Age', 'Position', 'Prospect_Prob', 'Prospect_Index', 'Offensive_IQ']
    print(final_list[output_cols].to_string(index=False))
    print("="*95)
    
    # 결과 CSV 저장
    final_list.to_csv('fw_prospect_top10.csv', index=False, encoding='utf-8-sig')
    print("💾 결과가 'fw_prospect_top10.csv'로 저장되었습니다.")
else:
    print("❌ 조건에 맞는 공격수 데이터가 없습니다. 포지션 명칭을 확인하세요.")

[14:13:56] Stdout logging level is INFO.
[14:13:56] Task: binary

[14:13:56] Start automl preset with listed constraints:
[14:13:56] - time: 300.00 seconds
[14:13:56] - CPU: 4 cores
[14:13:56] - memory: 16 GB

[14:13:56] Train data shape: (3019, 72)

[14:14:05] Layer 1 train process start. Time left 291.24 secs
[14:14:05] Selector_LightGBM fitting and predicting completed
[14:14:06] Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
[14:14:07] Fitting Lvl_0_Pipe_0_Mod_0_LightGBM finished. score = 0.8283300279176833
[14:14:07] Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
[14:14:07] Start fitting Lvl_0_Pipe_0_Mod_1_CatBoost ...
[14:14:13] Fitting Lvl_0_Pipe_0_Mod_1_CatBoost finished. score = 0.8507521604101569
[14:14:13] Lvl_0_Pipe_0_Mod_1_CatBoost fitting and predicting completed
[14:14:13] Time left 283.38 secs

[14:14:13] Layer 1 training completed.

[14:14:13] Blending: optimization starts with equal weights. Score = 0.8490395
[14:14:13] Blending: iteration 0: score = 0.85